In [1]:
import numpy as np
import itertools

# Generate a Random likelihood and priors

generates two random likelihoods -- named aLikelihood, bLikelihood
finds the joint probability distribution of the two likelihoods $P(D|A,B)$  
Sample randomly to get a random prior for each of the two events $P(A), P(B)$  
return $P(A), \: P(B), \: P(D|A,B)$

In [2]:
def setUpBayesianInference(eventAMeanSampleSpaceSize, eventBMeanSampleSpaceSize, randomSeed, isPriorUniform=False):
    np.random.seed(randomSeed)
    
    aLikelihood = constructRandomLikelihood(eventAMeanSampleSpaceSize, 'a')
    bLikelihood = constructRandomLikelihood(eventBMeanSampleSpaceSize, 'b')
    
    jointLikelihood = getJointLikelihood(aLikelihood, bLikelihood)
    
    sampleSpaceANames = list(aLikelihood.keys())
    sampleSpaceBNames = list(bLikelihood.keys())
    
    aPrior = constructPrior(sampleSpaceANames,isPriorUniform)
    bPrior = constructPrior(sampleSpaceBNames,isPriorUniform)
    
    return([aPrior,  bPrior, jointLikelihood])


## Helpers for generating likelihoods/priors

In [3]:
def constructRandomLikelihood(meanSampleSpaceSize, eventIDString):
    sampleSpaceSize = np.maximum(2, np.random.poisson(meanSampleSpaceSize))
    outcomeNames = [eventIDString+str(outcomeIndex) for outcomeIndex in range(sampleSpaceSize)]
    likelihood = {outcome: np.random.random() for outcome in outcomeNames}
    return(likelihood)

def getJointLikelihood(eventAPMF, eventBPMF):
    jointPMF = {(keya, keyb):eventAPMF[keya]*eventBPMF[keyb] for keya, keyb in itertools.product(eventAPMF.keys(), eventBPMF.keys())}
    return(jointPMF)

def constructPrior(namesOfAllOutcomes, isUniform = False):
    if isUniform:
        prior = {a: 1/len(namesOfAllOutcomes) for a in namesOfAllOutcomes}
    else:
        unnormalizedPrior = {outcome: np.random.random() for outcome in namesOfAllOutcomes}
        prior = normalizeDictionaryValues(unnormalizedPrior)
    return(prior)

def normalizeDictionaryValues(unnormalizedDictionary):
    totalSum = sum(unnormalizedDictionary.values())
    normalizedDictionary = {originalKey: val/totalSum for originalKey, val in unnormalizedDictionary.items()}
    return(normalizedDictionary)

# Find Posterior given the priors and likelihood

Uses Bayes law to find the joint posterior and then marginalizes to find marginal posteriors

$$P(A, B | D) \propto P(D|A,B)P(A, B)$$

$$ P(A|D) = \sum_B P(A,B|D) $$

In [4]:
def getPosterior(priorOfA, priorOfB, likelihood):
    normalizedPriorOfA = normalizeDictionaryValues(priorOfA)
    normalizedPriorOfB = normalizeDictionaryValues(priorOfB)
    
    unnormalizedJointPosterior = {(aEvent, bEvent): calculateOutcomePosterier(normalizedPriorOfA, normalizedPriorOfB, likelihood, aEvent, bEvent) \
                      for aEvent, bEvent in likelihood.keys()}
    jointPosterior = normalizeDictionaryValues(unnormalizedJointPosterior)
    
    marginalOfA = {keyOfA : getMarginalizedOutcomeProbability(keyOfA, jointPosterior) for keyOfA in normalizedPriorOfA.keys()}
    marginalOfB = {keyOfB : getMarginalizedOutcomeProbability(keyOfB, jointPosterior) for keyOfB in normalizedPriorOfB.keys()}
    
    return([marginalOfA, marginalOfB])

## Helper Functions

In [5]:
def getMarginalizedOutcomeProbability(outcome, jointPMF):
    probabilityOfOutcome = sum([jointPMF[(keyOne, keyTwo)] if (keyOne==outcome )or (keyTwo==outcome)  else 0 for (keyOne, keyTwo) in jointPMF.keys()])
    return(probabilityOfOutcome)

def calculateOutcomePosterier(priorOfA, priorOfB, likelihood, aEvent, bEvent):
    posterior = priorOfA[aEvent]*priorOfB[bEvent]*likelihood[(aEvent, bEvent)]
    return(posterior)

## Examples

### Setup the Inference

In [6]:
priorA, priorB, likelihood = setUpBayesianInference(eventAMeanSampleSpaceSize=5, 
                                                    eventBMeanSampleSpaceSize=5, 
                                                    randomSeed = 2)
print("Prior of A: ", priorA, "\n")
print("Prior of B: ",priorB, "\n")
print("Likelihood: ", likelihood, "\n")

Prior of A:  {'a0': 0.1901883045926098, 'a1': 0.8098116954073902} 

Prior of B:  {'b0': 0.3754696289252865, 'b1': 0.21730244836408819, 'b2': 0.3722099802058252, 'b3': 0.03501794250480023} 

Likelihood:  {('a0', 'b0'): 0.2703934662172901, ('a0', 'b1'): 0.23034740251572647, ('a0', 'b2'): 0.058585663805993414, ('a0', 'b3'): 0.22357205654580428, ('a1', 'b0'): 0.2611046640834065, ('a1', 'b1'): 0.2224342991632098, ('a1', 'b2'): 0.05657307582970412, ('a1', 'b3'): 0.21589170603670374} 



In [7]:
getPosterior(priorA, priorB, likelihood)

[{'a0': 0.19563062885653146, 'a1': 0.8043693711434686},
 {'b0': 0.560244192974484,
  'b1': 0.2762194753670806,
  'b2': 0.12033327206270723,
  'b3': 0.043203059595728226}]

### Example 1

In [8]:
lika = {'a0': .6, 'a1': .1}
likb = {'b0': .7, 'b1': .2}
jointLik = getJointLikelihood(lika, likb)
pa = {'a0': .5, 'a1': .5}
pb = {'b0': .25, 'b1': .75}

In [9]:
print(jointLik)

{('a0', 'b0'): 0.42, ('a0', 'b1'): 0.12, ('a1', 'b0'): 0.06999999999999999, ('a1', 'b1'): 0.020000000000000004}


In [10]:
getPosterior(pa, pb, jointLik)

[{'a0': 0.8571428571428571, 'a1': 0.14285714285714285},
 {'b0': 0.5384615384615384, 'b1': 0.46153846153846156}]

### Example 2

In [11]:
exampleTwoPriorofA = {'red': 1/10 , 'blue': 4/10, 'green': 2/10, 'purple': 3/10}
exampleTwoPriorofB = {'x': 1/5, 'y': 2/5, 'z': 2/5}

likA = {'red': .5 , 'blue': .2, 'green': .6, 'purple': .8}
likB = {'x': .4, 'y': .6, 'z': .8}


#exampleTwoLikelihood = getJointLikelihood(likA, likB)
exampleTwoLikelihood = {('red', 'x'): 0.2, ('red', 'y'): 0.3, ('red', 'z'): 0.4, ('blue', 'x'): 0.08, ('blue', 'y'): 0.12, ('blue', 'z'): 0.16, ('green', 'x'): 0.24, ('green', 'y'): 0.36, ('green', 'z'): 0.48, ('purple', 'x'): 0.32, ('purple', 'y'): 0.48, ('purple', 'z'): 0.64}

print(exampleTwoLikelihood)

{('red', 'x'): 0.2, ('red', 'y'): 0.3, ('red', 'z'): 0.4, ('blue', 'x'): 0.08, ('blue', 'y'): 0.12, ('blue', 'z'): 0.16, ('green', 'x'): 0.24, ('green', 'y'): 0.36, ('green', 'z'): 0.48, ('purple', 'x'): 0.32, ('purple', 'y'): 0.48, ('purple', 'z'): 0.64}


In [12]:
print(getPosterior(exampleTwoPriorofA, exampleTwoPriorofB, exampleTwoLikelihood))

[{'red': 0.10204081632653064, 'blue': 0.163265306122449, 'green': 0.24489795918367352, 'purple': 0.4897959183673469}, {'x': 0.125, 'y': 0.375, 'z': 0.5}]


### Testing Example

In [13]:
exampleThreePriorA = {'gold': 3/6, 'silver': 1/6, 'bronze': 2/6}
exampleThreePriorB = {'samoas': 3/20, 'tagalogs':2/20, 'mintthins': 14/20, 'lemondrops':1/20}

likMedal = {'gold': .5, 'silver': .9, 'bronze': .2}
likCookie = {'samoas': .8, 'tagalogs': .6, 'mintthins': .5 , 'lemondrops': .1}

exampleThreeLikelihood = {('gold', 'samoas'): 0.4, ('gold', 'tagalogs'): 0.3, ('gold', 'mintthins'): 0.25, ('gold', 'lemondrops'): 0.05, ('silver', 'samoas'): 0.72, ('silver', 'tagalogs'): 0.54, ('silver', 'mintthins'): 0.45, ('silver', 'lemondrops'): 0.09, ('bronze', 'samoas'): 0.16, ('bronze', 'tagalogs'): 0.12, ('bronze', 'mintthins'): 0.1, ('bronze', 'lemondrops'): 0.02}

In [14]:
print(exampleThreeLikelihood)

{('gold', 'samoas'): 0.4, ('gold', 'tagalogs'): 0.3, ('gold', 'mintthins'): 0.25, ('gold', 'lemondrops'): 0.05, ('silver', 'samoas'): 0.72, ('silver', 'tagalogs'): 0.54, ('silver', 'mintthins'): 0.45, ('silver', 'lemondrops'): 0.09, ('bronze', 'samoas'): 0.16, ('bronze', 'tagalogs'): 0.12, ('bronze', 'mintthins'): 0.1, ('bronze', 'lemondrops'): 0.02}


In [15]:
getPosterior(exampleThreePriorA, exampleThreePriorB, exampleThreeLikelihood)

[{'gold': 0.5357142857142856,
  'silver': 0.3214285714285714,
  'bronze': 0.14285714285714282},
 {'samoas': 0.22429906542056072,
  'tagalogs': 0.11214953271028037,
  'mintthins': 0.6542056074766355,
  'lemondrops': 0.009345794392523366}]